In [ ]:
import asyncio
import json
import aiohttp
import pandas as pd
import numpy as np
from understat import Understat

In [ ]:
session = aiohttp.ClientSession()
understat = Understat(session)
results = await understat.get_team_results(
    "Chelsea",
    2021,
)

In [ ]:
# save json response to variable
response = json.dumps(results)

# save json response into file
with open('understat.json', 'w') as json_response:
    json.dump(response, json_response)

In [ ]:
# load data using Python JSON module
with open('understat.json','r') as f:
    data = json.loads(f.read())
    
# convert json to pandas dataframe
df = pd.read_json(data)

In [ ]:
# clean up dictionaries
df_goals=df['goals'].apply(pd.Series).rename(columns={"h": "h_goals", "a": "a_goals"})
df_xG=df['xG'].apply(pd.Series).rename(columns={"h": "h_xG", "a": "a_xG"})
df_h=df['h'].apply(pd.Series).rename(columns={"id": "h_id", "title": "h_name", "short_title": "h_abbr"})
df_a=df['a'].apply(pd.Series).rename(columns={"id": "a_id", "title": "a_name", "short_title": "a_abbr"})

# concat columns and drop/remove the columns with dictionaries 
df = pd.concat([df, df_h, df_a, df_goals, df_xG], axis = 1).drop(columns=['isResult', 'h','a','goals','xG','forecast'], axis = 1)

# get Chelsea values for reporting purposes
df['che_goals'] = np.where(df['side']=='a',df['a_goals'],df['h_goals'])
df['che_xG'] = np.where(df['side']=='a',df['a_xG'],df['h_xG'])

# get opponent values for reporting purposes
df['opp_goals'] = np.where(df['h_abbr']=='CHE',df['a_goals'],df['h_goals'])
df['opp_xG'] = np.where(df['h_abbr']=='CHE',df['a_xG'],df['h_xG'])

df['gameweek'] = df.index+1

# save to csv
df.to_csv('che_xg.csv',index=False)

In [ ]:
# melt master dataframe to position data for 2 measures (goals, xG) with gameweek and team as dimension

# melt df for goals measures
df_melt_h_goals=df
df_melt_h_goals=df_melt_h_goals.melt(id_vars=['h_abbr'], value_vars=['h_goals'])
df_melt_h_goals['gameweek'] = df.index+1

df_melt_a_goals=df
df_melt_a_goals=df_melt_a_goals.melt(id_vars=['a_abbr'], value_vars=['a_goals'])
df_melt_a_goals['gameweek'] = df.index+1

goals_frames=[df_melt_h_goals,df_melt_a_goals]
goals_result=pd.concat(goals_frames)

goals_result['team'] = goals_result['h_abbr'].mask(pd.isnull, goals_result['a_abbr'])
goals_result=goals_result.drop(columns=['h_abbr', 'variable','a_abbr']).rename(columns={"value": "goals"})


# melt df for xG measures
df_melt_h_xG=df
df_melt_h_xG=df_melt_h_xG.melt(id_vars=['h_abbr'], value_vars=['h_xG'])
df_melt_h_xG['gameweek'] = df.index+1

df_melt_a_xG=df
df_melt_a_xG=df_melt_a_xG.melt(id_vars=['a_abbr'], value_vars=['a_xG'])
df_melt_a_xG['gameweek'] = df.index+1

xG_frames=[df_melt_h_xG,df_melt_a_xG]
xG_result=pd.concat(xG_frames)

xG_result['team'] = xG_result['h_abbr'].mask(pd.isnull, xG_result['a_abbr'])
xG_result=xG_result.drop(columns=['h_abbr', 'variable','a_abbr']).rename(columns={"value": "xG"})

In [ ]:
result=goals_result.merge(xG_result, on=['gameweek','team'], how='left')


# save to csv
result.to_csv('che.csv',index=False)